In [1]:
%load_ext autoreload
%autoreload 2
import numpy as np
import numpy.ma as ma
import matplotlib
import matplotlib.pyplot as plt
import matplotlib.cm as cm
import matplotlib.ticker as ticker
import matplotlib.dates as dates
from mpl_toolkits.axes_grid1 import ImageGrid,make_axes_locatable,host_subplot
from mpl_toolkits.basemap import Basemap
from datetime import datetime, timedelta
import sys
import os
import pyPIPS.utils as utils
import pyPIPS.thermolib as thermo
import pyPIPS.DSDlib as dsd
#import pyPIPS.disdrometer_module as dis
import pyPIPS.plotmodule as PIPSplot
import pyPIPS.simulator as sim
import pyPIPS.pips_io as pipsio
import pyPIPS.PIPS as pips
import pyPIPS.parsivel_params as pp
import pyPIPS.parsivel_qc as pqc
from pyCRMtools.modules import plotmodule as plotmod
from pyCRMtools.modules import utils as CRMutils
from pyCRMtools.pycaps import arps_read
from pyCRMtools.pycaps import pycaps_fields
from pyCRMtools.pycaps import calvars_radar as radar
import pandas as pd
import xarray as xr
import glob
import numpy.random as random
from scipy.stats import gamma, uniform
from scipy.special import gamma as gammafunc
from scipy import ndimage
from metpy.plots import StationPlot
from metpy.calc import wind_components
from metpy.cbook import get_test_data
from metpy.plots import StationPlot
from metpy.plots.wx_symbols import current_weather, sky_cover
from metpy.units import units
%matplotlib inline


## You are using the Python ARM Radar Toolkit (Py-ART), an open source
## library for working with weather radar data. Py-ART is partly
## supported by the U.S. Department of Energy as part of the Atmospheric
## Radiation Measurement (ARM) Climate Research Facility, an Office of
## Science user facility.
##
## If you use this software to prepare a publication, please cite:
##
##     JJ Helmus and SM Collis, JORS 2016, doi: 10.5334/jors.119



/Users/dawson29/miniconda3/envs/pyPIPS/lib/python3.7/site-packages/pyart/graph/cm.py:104: FutureWarning: elementwise comparison failed; returning scalar instead, but in the future will perform elementwise comparison
  if 'red' in spec:
/Users/dawson29/miniconda3/envs/pyPIPS/lib/python3.7/site-packages/pyart/graph/cm_colorblind.py:32: FutureWarning: elementwise comparison failed; returning scalar instead, but in the future will perform elementwise comparison
  if 'red' in spec:


In [2]:
# Define dictionaries, keyed by case date (i.e. '060509', '060709', '060909', '033116'), to store parameters related
# to NEXRAD radar data, disdrometer data, and model output, respectively

# Case we are looking at right now. Should only have to change this up here and then execute all the cells below
# to generate the appropriate analysis
casedate = '033116'

# Import the file containing the dictionaries needed to gather the radar, disdrometer, and model data.
sys.path.append('/Users/dawson29/Dropbox/Projects/VORTEXSE/vortexse_enkf_dsd_study/configs/2016_IOP3')

from PIPSsim_1km_dicts import *

In [3]:
# Read in the PIPS data for the first probe for testing
dis_dict = init_dis_dict[casedate]
dirname = dis_dict['dis_dir']

conv_df_list = []
parsivel_df_list = []
vd_matrix_da_list = []

for i, PIPS_name in enumerate(dis_dict['dis_names']):
    print(PIPS_name)
    filename = dis_dict['disfilenames'][i]
    filepath = os.path.join(dirname, filename)
    print("Reading file: {}".format(filepath))
    conv_df, parsivel_df, vd_matrix_da = pipsio.read_PIPS(filepath)
    conv_df_list.append(conv_df)
    parsivel_df_list.append(parsivel_df)
    vd_matrix_da_list.append(vd_matrix_da)

PIPS1A
Reading file: /Users/ddawson/Dropbox/Projects/VORTEXSE/obs_data/PIPS/2016/IOP3/PIPS_1A_IOP_3_D1.txt
GPS time: Thu Mar 31 22:12:03 2016, Logger time: Thu Mar 31 22:12:00 2016
GPS Offset: 0:00:03
PIPS1B
Reading file: /Users/ddawson/Dropbox/Projects/VORTEXSE/obs_data/PIPS/2016/IOP3/PIPS_1B_IOP_3_D1.txt
GPS time: Thu Mar 31 22:01:33 2016, Logger time: Thu Mar 31 22:01:30 2016
GPS Offset: 0:00:03
PIPS2A
Reading file: /Users/ddawson/Dropbox/Projects/VORTEXSE/obs_data/PIPS/2016/IOP3/PIPS_2A_IOP_3_D1.txt
GPS time: Thu Mar 31 22:25:03 2016, Logger time: Thu Mar 31 22:25:00 2016
GPS Offset: 0:00:03
PIPS2B
Reading file: /Users/ddawson/Dropbox/Projects/VORTEXSE/obs_data/PIPS/2016/IOP3/PIPS_2B_IOP_3_D1.txt
GPS time: Thu Mar 31 21:48:23 2016, Logger time: Thu Mar 31 21:48:20 2016
GPS Offset: 0:00:03


In [4]:
# print(vd_matrix_da)
#print(spectrum_da.loc['2016-03-31T22:30'])
diameter_bins = pp.parsivel_parameters['avg_diameter_bins_mm']
# print(diameter_bins)
fallspeed_bins = pp.parsivel_parameters['avg_fallspeed_bins_mps']
# print(fallspeed_bins)

empirical_fallspeed = pips.calc_empirical_fallspeed(diameter_bins)
# print(empirical_fallspeed)

fallspeed_spectrum = pips.calc_fallspeed_spectrum(diameter_bins, fallspeed_bins, use_measured_fallspeed=True)
print(fallspeed_spectrum)

<xarray.DataArray (fallspeed_bin: 32, diameter_bin: 32)>
array([[ 0.05,  0.05,  0.05, ...,  0.05,  0.05,  0.05],
       [ 0.15,  0.15,  0.15, ...,  0.15,  0.15,  0.15],
       [ 0.25,  0.25,  0.25, ...,  0.25,  0.25,  0.25],
       ...,
       [15.2 , 15.2 , 15.2 , ..., 15.2 , 15.2 , 15.2 ],
       [17.6 , 17.6 , 17.6 , ..., 17.6 , 17.6 , 17.6 ],
       [20.8 , 20.8 , 20.8 , ..., 20.8 , 20.8 , 20.8 ]])
Coordinates:
    fallspeed  (fallspeed_bin) float64 0.05 0.15 0.25 0.35 ... 15.2 17.6 20.8
    diameter   (diameter_bin) float64 0.0625 0.1875 0.3125 ... 19.0 21.5 24.5
Dimensions without coordinates: fallspeed_bin, diameter_bin


In [5]:
# Do some QC
vd_matrix_QC_da_list = []
for vd_matrix_da in vd_matrix_da_list:
    vd_matrix_QC = pqc.strongwindQC(vd_matrix_da)
    vd_matrix_QC = pqc.rainonlyQC(vd_matrix_QC)
    vd_matrix_QC_da_list.append(vd_matrix_QC)

Severe Wind contamination, masking entire PSD!
Severe Wind contamination, masking entire PSD!
Severe Wind contamination, masking entire PSD!
Severe Wind contamination, masking entire PSD!
380
Severe Wind contamination, masking entire PSD!
495
Severe Wind contamination, masking entire PSD!
Severe Wind contamination, masking entire PSD!
265
Severe Wind contamination, masking entire PSD!
634


In [6]:
# Calculate ND
ND_list = []

for vd_matrix_QC in vd_matrix_QC_da_list:
    ND = pips.calc_ND(vd_matrix_QC.where(vd_matrix_QC > 0.0), fallspeed_spectrum, 10.)
    ND_list.append(ND)

fallspeed_spectrum <xarray.DataArray (fallspeed_bin: 32, diameter_bin: 32)>
array([[ 0.05,  0.05,  0.05, ...,  0.05,  0.05,  0.05],
       [ 0.15,  0.15,  0.15, ...,  0.15,  0.15,  0.15],
       [ 0.25,  0.25,  0.25, ...,  0.25,  0.25,  0.25],
       ...,
       [15.2 , 15.2 , 15.2 , ..., 15.2 , 15.2 , 15.2 ],
       [17.6 , 17.6 , 17.6 , ..., 17.6 , 17.6 , 17.6 ],
       [20.8 , 20.8 , 20.8 , ..., 20.8 , 20.8 , 20.8 ]])
Coordinates:
    fallspeed  (fallspeed_bin) float64 0.05 0.15 0.25 0.35 ... 15.2 17.6 20.8
    diameter   (diameter_bin) float64 0.0625 0.1875 0.3125 ... 19.0 21.5 24.5
Dimensions without coordinates: fallspeed_bin, diameter_bin
vd_matrix <xarray.DataArray 'velocity-diameter matrix' (time: 380, fallspeed_bin: 32, diameter_bin: 32)>
array([[[nan, nan, nan, ..., nan, nan, nan],
        [nan, nan, nan, ..., nan, nan, nan],
        [nan, nan, nan, ..., nan, nan, nan],
        ...,
        [nan, nan, nan, ..., nan, nan, nan],
        [nan, nan, nan, ..., nan, nan, nan],
   

In [156]:
DSD_interval = 10.
bin_width = ND_list[-1]['max_diameter'] - ND_list[-1]['min_diameter']
D0_list = []
rainrate_list = []
for conv_df, parsivel_df, vd_matrix_QC, ND in zip(conv_df_list, parsivel_df_list, vd_matrix_QC_da_list, ND_list):

    conv_df = pips.calc_thermo(conv_df)
    # Resample to PIPS intervals and times
    
    PSD_datetimes = pips.get_PSD_datetimes(vd_matrix_QC)
    sec_offset = PSD_datetimes[0].second
    conv_resampled_df = pips.resample_conv('PIPS', DSD_interval, sec_offset, conv_df)
    conv_resampled_df_index = conv_resampled_df.index.intersection(parsivel_df.index)
    conv_resampled_df = conv_resampled_df.loc[conv_resampled_df_index]

    # Compute empirical fallspeed curves
    fallspeeds = pips.calc_empirical_fallspeed(diameter_bins, correct_rho=True, rho=conv_resampled_df['rho'])
    print(ND.shape, fallspeeds.shape)
    rainrate_bin = (6. * 10.**-4.) * np.pi * fallspeeds * diameter_bins**3. * ND * bin_width
    rainrate = rainrate_bin.sum(dim='diameter_bin')
    
    # Compute D0
    D0 = dsd.calc_D0_bin(ND) * 1000.
    D0 = D0.fillna(0.0)
    
    print(D0)
    print(rainrate)
    
    rainrate_list.append(rainrate)
    D0_list.append(D0)
    

(380, 32) (380, 32)
<xarray.DataArray (time: 380)>
array([0.        , 0.        , 0.        , 0.        , 0.        ,
       0.        , 0.        , 0.        , 0.        , 0.        ,
       0.        , 0.        , 0.3125    , 0.        , 0.        ,
       0.        , 0.        , 0.        , 0.        , 0.        ,
       0.        , 0.        , 0.        , 0.        , 0.        ,
       0.        , 0.        , 0.        , 0.        , 0.        ,
       0.        , 0.        , 0.        , 0.        , 0.        ,
       0.        , 0.        , 0.        , 0.        , 0.        ,
       0.        , 0.        , 0.        , 0.        , 0.        ,
       0.        , 0.        , 0.        , 0.        , 0.        ,
       0.        , 0.        , 0.        , 0.        , 0.        ,
       0.        , 0.        , 0.        , 0.        , 0.        ,
       0.        , 0.        , 0.        , 0.        , 0.        ,
       0.        , 0.        , 0.        , 0.        , 0.        ,
       0.  

<xarray.DataArray (time: 634)>
array([0.        , 0.        , 0.        , 0.        , 0.        ,
       0.        , 0.        , 0.        , 0.        , 0.        ,
       0.        , 0.        , 0.        , 0.        , 0.        ,
       0.        , 0.        , 0.        , 0.        , 0.        ,
       0.        , 0.        , 0.        , 0.        , 0.        ,
       0.        , 0.        , 0.        , 0.        , 0.        ,
       0.        , 0.        , 0.        , 0.        , 0.        ,
       0.        , 0.        , 0.        , 0.        , 0.        ,
       0.        , 0.        , 0.        , 0.        , 0.        ,
       0.        , 0.        , 0.        , 0.        , 0.        ,
       0.        , 0.        , 0.        , 0.        , 0.        ,
       0.        , 0.        , 0.        , 0.        , 0.        ,
       0.        , 0.        , 0.        , 0.        , 0.        ,
       0.        , 0.        , 0.        , 0.        , 0.        ,
       0.        , 0.        , 

In [165]:
D0 = dsd.calc_D0_bin(ND_list[0]) * 1000.

<xarray.DataArray (time: 380)>
array([ 0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  2,  0,  0,  0,  0,
        0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,
        0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,
        0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,
        0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,
        0,  0,  0,  0,  0,  0,  0,  0,  0,  5,  7,  7,  8,  7, 10, 10, 11,
       11, 10, 10, 12, 11, 15, 14, 12, 12, 10, 12, 11, 10, 13, 14, 14, 14,
       14, 14, 14, 14, 14, 15, 14, 15, 14, 14, 15, 14, 14, 15, 15, 13, 11,
       12, 11, 11, 11, 10,  9,  9, 10, 10, 11, 13, 14, 13, 12, 11, 12, 12,
       11, 11, 13, 14, 13, 14, 15, 15, 15, 15, 15, 15, 14, 15, 14, 15, 14,
       13, 14, 13, 14, 13, 13, 14, 13, 13, 14, 14, 15, 14, 14, 15, 14, 13,
       13, 13, 14, 14, 15, 14, 15, 15, 14, 16, 16, 15, 14, 15, 15, 15, 14,
       15, 15, 16, 14, 15, 14,  0,  0,  0,  0, 13, 15, 15, 14, 15, 15

In [132]:
# Set up D0 and RR bins for SATP
# Now set up D0 and RR bins for the SATP procedure
D0_bins = np.arange(0., 6.05, 0.05)
# Following is from
# https://stackoverflow.com/questions/45234987/numpy-range-created-using-percentage-increment
# TODO: make a function out of this and put in pyPIPS.utils
RR_start = 0.1
RR_stop = 250.
RR_pct_incr = 10.
RR_incr = (100. + RR_pct_incr) / 100.
RR_bins = RR_start * np.full(int(np.log(RR_stop / RR_start) / np.log(RR_incr)),
                             RR_incr).cumprod()

In [155]:
ND2_list = []

for D0, rainrate, ND in zip(D0_list, rainrate_list, ND_list):
    ND2 = ND.copy()
    #print(ND2)
    # GOTCHA: need to explicitly add dimension name to new coordinate, otherwise xarray changes the dimensions
    # of the *existing* coordinates all to 'time'. This seems like a bug. May raise it on the github.
    print(D0)
    ND2.coords['D0'] = ('time', D0)
    ND2.coords['RR'] = ('time', rainrate)
    #print(ND2)
    
    D0_indices = np.digitize(D0, D0_bins)
    RR_indices = np.digitize(rainrate, RR_bins)
    ND2.coords['D0_RR'] = ('time', pd.MultiIndex.from_arrays([D0_indices, RR_indices], names=['D0_idx', 'RR_idx']))
    ND2_list.append(ND2)



<xarray.DataArray (time: 380)>
array([0.        , 0.        , 0.        , 0.        , 0.        ,
       0.        , 0.        , 0.        , 0.        , 0.        ,
       0.        , 0.        , 0.3125    , 0.        , 0.        ,
       0.        , 0.        , 0.        , 0.        , 0.        ,
       0.        , 0.        , 0.        , 0.        , 0.        ,
       0.        , 0.        , 0.        , 0.        , 0.        ,
       0.        , 0.        , 0.        , 0.        , 0.        ,
       0.        , 0.        , 0.        , 0.        , 0.        ,
       0.        , 0.        , 0.        , 0.        , 0.        ,
       0.        , 0.        , 0.        , 0.        , 0.        ,
       0.        , 0.        , 0.        , 0.        , 0.        ,
       0.        , 0.        , 0.        , 0.        , 0.        ,
       0.        , 0.        , 0.        , 0.        , 0.        ,
       0.        , 0.        , 0.        , 0.        , 0.        ,
       0.        , 0.        , 

In [166]:
print(ND_list[0])
print(ND2_list[0])
ND2 = ND2_list[0].reset_index('time')
print(ND2)
ND2_2 = ND2_list[1].reset_index('time')
ND2_combined = xr.concat([ND2, ND2_2], dim='time')
print(ND2_combined)

print(ND2_combined.groupby('D0_RR').count(dim='time')[:, 0])

# ND2_groups = []
# ND2_combined = xr.merge(ND2_list)
# print(ND2_combined)

ND2_list_2 = []

# for ND2 in ND2_list:
# #     print(D0_indices)
# #     print([D0_bins[x[0]] for x,_ in list(ND2.groupby('D0_RR'))])
# #     print([RR_bins[x[1]] for x,_ in list(ND2.groupby('D0_RR'))])
# #     print([y.sizes['time'] for x,y in list(ND2.groupby('D0_RR'))])
#     ND2 = ND2.reset_index('time')
#     ND2_group = ND2.groupby('D0_RR')
#     ND2_groups.append(ND2.groupby('D0_RR'))
#     print(ND2.groupby('D0_RR').count(dim='time')[:, 0])
    

<xarray.DataArray (time: 380, diameter_bin: 32)>
array([[0., 0., 0., ..., 0., 0., 0.],
       [0., 0., 0., ..., 0., 0., 0.],
       [0., 0., 0., ..., 0., 0., 0.],
       ...,
       [0., 0., 0., ..., 0., 0., 0.],
       [0., 0., 0., ..., 0., 0., 0.],
       [0., 0., 0., ..., 0., 0., 0.]])
Coordinates:
  * time           (time) datetime64[ns] 2016-03-31T22:11:53 ... 2016-03-31T23:15:03
    diameter       (diameter_bin) float64 0.0625 0.1875 0.3125 ... 21.5 24.5
    min_diameter   (diameter_bin) float64 0.0 0.125 0.25 ... 18.0 20.0 23.0
    max_diameter   (diameter_bin) float64 0.125 0.25 0.375 ... 20.0 23.0 26.0
    flagged times  (time) int64 0 0 0 0 0 0 0 0 0 0 0 ... 0 0 0 0 0 0 0 0 0 0 0
Dimensions without coordinates: diameter_bin
<xarray.DataArray (time: 380, diameter_bin: 32)>
array([[0., 0., 0., ..., 0., 0., 0.],
       [0., 0., 0., ..., 0., 0., 0.],
       [0., 0., 0., ..., 0., 0., 0.],
       ...,
       [0., 0., 0., ..., 0., 0., 0.],
       [0., 0., 0., ..., 0., 0., 0.],
     

In [ ]:
# Compute sigma and Dm
D = ND_list[0]['diameter']
dD = ND_list[0]['max_diameter'] - ND_list[0]['min_diameter']

Dm_list = []
sigma_list = []

for ND in ND_list:
    Dm = dsd.calc_Dmpq_binned(4, 3, ND)
    sigma = dsd.calc_sigma(D, dD, ND)
    
    Dm_list.append(Dm)
    sigma_list.append(sigma)

In [ ]:
# Plot sigma vs Dm
fig, ax = plt.subplots(figsize=(10, 10))
sc_list = []

for Dm, sigma in zip(Dm_list, sigma_list):
    sc = ax.scatter(Dm * 1000., sigma * 1000.)
    sc_list.append(sc)
ax.set_xlim(0.0, 6.0)
ax.set_ylim(0.0, 3.5)

In [ ]:
DSD_fit = xr.open_dataset('/Users/dawson29/Dropbox/Projects/VORTEXSE/obs_data/PIPS/2016/IOP3/DSD_fits_PIPS2B_60s_20160331220000_20160331234000.nc')

In [ ]:
conv_df = pips.calc_thermo(conv_df)

DSD_interval = 10.
PSD_datetimes = pips.get_PSD_datetimes(vd_matrix_da)
PSD_datetimes_dict = pips.get_PSD_time_bins(PSD_datetimes)
PSDstarttimes = PSD_datetimes_dict['PSD_datetimes_edges']
PSDmidtimes = PSD_datetimes_dict['PSD_datetimes']

sec_offset = PSD_datetimes[0].second
conv_resampled_df = pips.resample_conv('PIPS', DSD_interval, sec_offset, conv_df)
conv_resampled_df_index = conv_resampled_df.index.intersection(parsivel_df.index)
conv_resampled_df = conv_resampled_df.loc[conv_resampled_df_index]

print(parsivel_df.columns)
#print(conv_resampled_df)

#parsivel_resampled_df = pips.resample_parsivel(60., parsivel_df)
resample_interval = 60.
intervalstr = '{:d}S'.format(int(resample_interval))
parsivel_resampler = parsivel_df.resample(intervalstr, label='right', closed='right',
                                          base=sec_offset)

parsivel_resampled_df = parsivel_resampler.agg({'precipintensity': np.mean, 'precipaccum': np.sum,
                                   'parsivel_dBZ': np.mean, 'pcount': np.sum,
                                   'signal_amplitude': np.mean, 'pvoltage': np.mean, 'sensor_temp': np.mean,
                                   'sample_interval': np.mean})

print(parsivel_resampled_df.columns)


# for time in vd_matrix_da['time'].to_index():
#     print(parsivel_df['pcount'].loc[time])
#     # print(conv_resampled_df['rho'].loc[time])
#     # print(vd_matrix_da.loc[time])

In [ ]:
# Do some QC
vd_matrix_QC = pqc.strongwindQC(vd_matrix_da)
vd_matrix_QC = pqc.rainonlyQC(vd_matrix_QC)
vdQC = vd_matrix_QC.loc['2016-03-31T22:30:03']
vdQC['diameter'] = ('diameter_bin', diameter_bins)
vdQC['fallspeed'] = ('fallspeed_bin', fallspeed_bins)
vdQC.where(vdQC > 0.0).plot(x='diameter', y='fallspeed', xlim=[0., 15], ylim=[0., 15.])
plt.plot(diameter_bins, empirical_fallspeed, color='r')

In [ ]:
# Calculate ND
ND = pips.calc_ND(vd_matrix_QC.where(vd_matrix_QC > 0.0), fallspeed_spectrum, 10.)

In [ ]:
timelimits = [PSDstarttimes[0], PSDstarttimes[-1]]
diamlimits = [0.0, 9.0]
diamytick = 1.0
DSDtype = 'observed'
locator = dates.MinuteLocator(byminute=[0, 15, 30, 45])
minorlocator = dates.MinuteLocator(byminute=range(0, 60, 5))
dateformat = '%H:%M'
formatter = dates.DateFormatter(dateformat)

axparams = {'majorxlocator': locator, 'majorxformatter': formatter,
            'minorxlocator': minorlocator,
            'axeslimits': [timelimits, diamlimits],
            'majorylocator': ticker.MultipleLocator(base=diamytick),
            'axeslabels': [None, 'D (mm)']}

disvars = {
    'min_diameter': ND['min_diameter'], 
    'PSDstarttimes': PSDstarttimes,
    'PSDmidtimes': PSDmidtimes, 
    'logND': np.log10(ND).T
}
dis_plot_name = 'PIPS1A'
PIPSplot.plotDSDmeteograms(dis_plot_name, './', axparams, disvars)

In [ ]:
np.log10(ND).T.plot(y='diameter', ylim=[0.0, 9.0], vmin=-1.0, vmax=3.0, aspect=5, size=3)

In [ ]:
# Now resample the vd matrix across time to go from 10-s to 60-s intervals
resample_interval = pips.check_requested_resampling_interval(60., 10.)
vd_matrix_rs = pips.resample_vd_matrix(resample_interval, vd_matrix_QC)
print(vd_matrix_QC['time'][:10])
print(vd_matrix_rs['time'][:10])

In [ ]:
ND_resampled = pips.calc_ND(vd_matrix_rs.where(vd_matrix_rs > 0.0), fallspeed_spectrum, 60.)
np.log10(ND_resampled).T.plot(y='diameter', ylim=[0.0, 9.0], vmin=-1.0, vmax=3.0, aspect=5, size=3)



In [ ]:
# Add additional thermodynamic variables to conv_df
conv_df = pips.calc_thermo(conv_df)

In [ ]:
# Resample conventional data to 60-s
datetime_range_onesec = conv_df.index.to_pydatetime()
sec_offset = pips.get_offset_seconds(datetime_range_onesec)
conv_rs_df = pips.resample_conv('PIPS', 60., sec_offset, conv_df)

In [ ]:
print(ND_resampled)

In [ ]:
# Compute sigma and Dm
Dm = dsd.calc_Dmpq_binned(4, 3, ND_resampled)
D = ND_resampled['diameter']
dD = ND_resampled['max_diameter'] - ND_resampled['min_diameter']

# D_m = D / 1000.
# dD_m = dD / 1000.
# Dm_m = Dm

# # print(D_m * 1000.)
# # print(dD_m * 1000.)
# # print(Dm_m * 1000.)

# ND_mneg4 = ND_resampled * 1000.

# sigma_numerator_bin = (D_m - Dm_m)**2. * ND_mneg4 * D_m**3. * dD_m
# #print(sigma_numerator_bin)
# sigma_numerator = sigma_numerator_bin.sum(dim='diameter_bin') # np.sum(sigma_numerator_bin, axis=0)
# #print(sigma_numerator)

# M3, _ = dsd.calc_moment_bin(ND_resampled, moment=3)
# #print(M3)


# sigma = np.sqrt(sigma_numerator / M3)
# print(sigma)

sigma = dsd.calc_sigma(D, dD, ND_resampled)

In [ ]:
# Plot sigma vs Dm
fig, ax = plt.subplots(figsize=(10, 10))
sc = ax.scatter(Dm * 1000., sigma * 1000.)
# ax.set_xlim(0.0, 3.5)
# ax.set_ylim(0.0, 2.0)

In [ ]:
M2, _ = dsd.calc_moment_bin(ND_resampled, moment=2)
M3, _ = dsd.calc_moment_bin(ND_resampled, moment=3)
M4, _ = dsd.calc_moment_bin(ND_resampled, moment=4)
M6, _ = dsd.calc_moment_bin(ND_resampled, moment=6)

DSD_MM24 = dsd.fit_DSD_MM24(M2, M4)
DSD_MM36 = dsd.fit_DSD_MM36(M3, M6)
DSD_MM346 = dsd.fit_DSD_MM346(M3, M4, M6)
DSD_MM246 = dsd.fit_DSD_MM246(M2, M4, M6)
DSD_MM234 = dsd.fit_DSD_MM234(M2, M3, M4)

D_min, D_max = dsd.get_max_min_diameters(ND_resampled)
print(D_max)

DSD_TMM246 = dsd.fit_DSD_TMM_xr(M2, M4, M6, D_min, D_max)

In [ ]:
ND_MM24 = dsd.calc_binned_DSD_from_params(*DSD_MM24, ND_resampled['diameter'])
ND_MM36 = dsd.calc_binned_DSD_from_params(*DSD_MM36, ND_resampled['diameter'])
ND_MM234 = dsd.calc_binned_DSD_from_params(*DSD_MM234, ND_resampled['diameter'])
ND_MM246 = dsd.calc_binned_DSD_from_params(*DSD_MM246, ND_resampled['diameter'])
ND_MM346 = dsd.calc_binned_DSD_from_params(*DSD_MM346, ND_resampled['diameter'])
ND_TMM246 = dsd.calc_binned_DSD_from_params(*DSD_TMM246, ND_resampled['diameter'])

In [ ]:
# Test plotting of individual DSD
time_to_plot = '2016-03-31T22:31:53'

axdict = {
    'xbin_left': ND_resampled['min_diameter'],
    'xbin_mid': ND_resampled['diameter'], 
    'xbin_right': ND_resampled['max_diameter'],
    'xlim': (0.0, 9.0), 
    'ylim': (10.**2., 10.**8.5), 
    'interval': int(resample_interval),
    'dis_name': 'PIPS_1A'
}

#print(ND_resampled['time'])

PSDdict = {
    'ND': ND_resampled.loc[time_to_plot]
}

PSDfitdict = {
    #'Exponential_24': (ND_MM24.loc[time_to_plot], 'Exp fit (MM24)'),
    #'Exponential_36': (ND_MM36.loc[time_to_plot], 'Exp fit (MM36)'),
    #'Gamma_234': (ND_MM234.loc[time_to_plot], 'Gamma fit (MM234)'),
    'Gamma_246': (ND_MM246.loc[time_to_plot], 'Gamma fit (MM246)'),
    #'Gamma_346': (ND_MM346.loc[time_to_plot], 'Gamma fit (MM346)')
    'TruncGamma_246': (ND_TMM246.loc[time_to_plot], 'Truncated Gamma fit (TMM246)'),
}

time_to_plot_datetime = pd.to_datetime(time_to_plot).to_pydatetime()

print(DSD_TMM246[0].loc[time_to_plot])

In [ ]:
import pyPIPS.timemodule as tm
xbin_left = axdict.get('xbin_left', np.empty((0)))
xbin_right = axdict.get('xbin_right', np.empty((0)))
xbin_mid = axdict.get('xbin_mid', np.empty((0)))
ND = PSDdict.get('ND', np.empty((0)))
# ND_onedrop = PSDdict.get('ND_onedrop', N.empty((0)))
interval = axdict.get('interval', 10)
dis_name = axdict.get('dis_name', None)

fig1 = plt.figure(figsize=(8, 6))
ax1 = fig1.add_subplot(111)
plt.title('{0:d}-s DSD fits for time {1} EST'.format(interval, time_to_plot_datetime.strftime(tm.timefmt2)))
ax1.bar(xbin_left, ND * 1000.0, xbin_right - xbin_left, 10.**2., align='edge', log=True, color='tan', 
        edgecolor='k')

for fitname, ND_tuple in PSDfitdict.items():
    if ((fitname == 'Dis Retr') | (fitname == 'Rad Retr')):
        ND_fit = ND_tuple[0] * 1000.
    else:
        ND_fit = ND_tuple[0]
#         print ND_fit
    label = ND_tuple[1]
    if(ND_fit.size):
        ax1.plot(xbin_mid, ND_fit, lw=2, label=label)

plt.legend(loc='upper left', numpoints=1, ncol=1, fontsize=8)
        
ax1.set_yscale('log')
ax1.set_ylim(10.**2.0, 10.**8.5)
ax1.set_xlim(0.0, 9.0)
ax1.set_xlabel('D (mm)')
ax1.set_ylabel(r'N(D) $(m^{-4})$')


In [ ]:
# Test merging of fit DataArrays to a Dataset

#print(DSD_MM24[0])

DSD_MM24[0].name = 'N0'
DSD_MM24[1].name = 'lamda'
# DSD_MM24[2].name = 'alpha'

DSD_MM24_da = xr.concat(DSD_MM24[:2], pd.Index(['N0', 'lamda'], name='parameter'))
DSD_MM24_da.name = 'DSD_MM24'
#print(DSD_MM24_da)

# DSD_MM24_ds = xr.merge(DSD_MM24[:2])

DSD_MM36[0].name = 'N0'
DSD_MM36[1].name = 'lamda'
DSD_MM36_da = xr.concat(DSD_MM36[:2], pd.Index(['N0', 'lamda'], name='parameter'))
DSD_MM36_da.name = 'DSD_MM36'
#print(DSD_MM36_da)

# DSD_MM36_ds = xr.merge(DSD_MM36[:2])

# fits_ds = xr.concat([DSD_MM24_da, DSD_MM36_da], dim='fit')

names = ['DSD_MM24', 'DSD_MM36']
data_arrays = [DSD_MM24_da, DSD_MM36_da]

fits_ds = xr.Dataset({name: da for name, da in zip(names, data_arrays)})
print(fits_ds)

# fits_ds = xr.merge([DSD_MM24, DSD_MM36, DSD_MM234, DSD_MM246, DSD_MM346, DSD_TMM246])
# print(fits_ds)


In [ ]:
conv_df, parsivel_df, _ = pipsio.read_PIPS(filepath)
parsivel_ds = parsivel_df.to_xarray()

In [ ]:
DSD_MM246 = fits_ds['DSD_MM']

In [ ]:
# Test converting conventional data DataFrame to xarray

print(conv_df)
conv_ds = conv_df.to_xarray()
conv_ds


In [ ]:
conv_ds = pipsio.conv_df_to_ds(conv_df)
print(conv_ds)
print(conv_ds['GPS_spd'])

In [ ]:
conv_nc_path = './conv_test.nc'
pipsio.dump_conv_netcdf(conv_nc_path, conv_ds)
conv_ds_from_netcdf.close()
conv_ds_from_netcdf = xr.open_dataset(conv_nc_path)
print(conv_ds_from_netcdf)
print(conv_ds_from_netcdf['time'])

In [ ]:
print(conv_ds_from_netcdf['GPS_time'])

In [ ]:
_, parsivel_df, vd_matrix_da = pipsio.read_PIPS(filepath)
parsivel_ds = pipsio.parsivel_df_to_ds(parsivel_df)

In [ ]:
parsivel_ds = pipsio.combine_parsivel_data(parsivel_ds, vd_matrix_da)
parsivel_ds